In [ ]:
# Read queries from a json file. Note that this is for demo only
import json

with open("test/queries.json", "r") as file:
    queries = json.load(file)

In [ ]:
# Load the catalog. For more details, refer to https://github.com/tokern/dbcat
# Provide credentials of the postgres database where catalog is stored.
# Note that you should have already created the database and user 

from dbcat import catalog_connection

catalog_conf = """
catalog:
  type: postgresql
  user: catalog_user
  password: catal0g_passw0rd
  host: 127.0.0.1
  port: 5432
  database: tokern
"""
catalog = catalog_connection(catalog_conf)

In [ ]:
# Scan the catalog from a file. Note that this is for demo only.
# Use dbcat (https://github.com/tokern/dbcat) to load the catalog from a database
# or a data warehouse

import json
with open("test/catalog.json", "r") as file:
    content = json.load(file)

source = catalog.add_source(name=content["name"], type=content["type"])
for s in content["schemata"]:
    schema = catalog.add_schema(s["name"], source=source)

    for t in s["tables"]:
        table = catalog.add_table(t["name"], schema)

        index = 0
        for c in t["columns"]:
            catalog.add_column(
                column_name=c["name"],
                type=c["type"],
                sort_order=index,
                table=table,
            )
            index += 1



In [ ]:
from data_lineage.parser import parse_queries, visit_dml_queries

# Parse all queries
parsed = parse_queries(queries)

# Visit the parse trees to extract source and target queries
visited = visit_dml_queries(catalog, parsed)

In [ ]:
# Create a graph and visualize it

from data_lineage.parser import create_graph
graph = create_graph(catalog, visited)

import plotly
plotly.offline.iplot(graph.fig())

In [ ]:
# Visualize a sub graph 

target = catalog.get_column(database_name="test",
                           schema_name="default",
                           table_name="normalized_pagecounts",
                           column_name="page_title")

sub_graph = graph.sub_graph(target)
sub_fig = sub_graph.fig()
plotly.offline.iplot(sub_fig)
